<a href="https://colab.research.google.com/github/mr7495/covid19/blob/master/data_Loading_Training_Evaluating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Aug 22 22:56:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#Connect your Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/INVESTIGACION/COURSES/UTEC/PROYECTO/COVID/CLASIFICACION/GRUPO-COVID/CODIGOS/CRISTIAN-LAZO/covid19/

/content/drive/MyDrive/INVESTIGACION/COURSES/UTEC/PROYECTO/COVID/CLASIFICACION/GRUPO-COVID/CODIGOS/CRISTIAN-LAZO/covid19


In [4]:
#upgrade and install essential libraries
#!pip install keras --upgrade
!pip install zipfile36
!pip install pydicom

     |████████████████████████████████| 1.9 MB 6.9 MB/s 


In [5]:
import keras
import numpy as np
import cv2
import os
import random
import shutil
import pandas as pd
import csv
import pydicom as dicom
import zipfile
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal
from tqdm import tqdm
import keras.backend as k
k.clear_session() #Clear keras backend 
import tensorflow as tf

In [6]:
fold_num=1 #Select Fold Number

In [7]:
#Here we set the data generators for applying data augmentation methods
train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rescale=1./255,zoom_range=0.05,rotation_range=360,width_shift_range=0.05,height_shift_range=0.05,shear_range=0.05)
test_datagen = ImageDataGenerator(rescale=1./255)
test_df = pd.read_csv("s_test.csv")

In [8]:
import keras.backend as k
k.clear_session() #Clear keras backend 
try:
  os.mkdir('models')
except:
  pass
full_name='concatenate'
classes_number=3 #Number of classes

from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.densenet import DenseNet121
#base_model = DenseNet121(weights='imagenet', include_top=False,input_shape=(HEIGHT, WIDTH,3))

input_tensor=Input(shape=(300,300,3))
######################################################################################################
base_model1 = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)
features1 = base_model1.output
######################################################################################################
base_model2 = ResNet50V2(weights='imagenet', include_top=False, input_tensor=input_tensor)
#base_model2 = DenseNet121(weights='imagenet', include_top=False,input_shape=input_tensor)
features2 = base_model2.output

concatenated=keras.layers.concatenate([features1,features2]) #Concatenate the extracted features
####################################################################################################
conv=keras.layers.Conv2D(1024, (1, 1),padding='same')(concatenated) #add the concatenated features to a convolutional layer
feature = Flatten(name='flatten')(conv)
dp = Dropout(0.5)(feature) #add dropout
preds = Dense(classes_number, activation='softmax', kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dp) 

Concatenated_model = Model(inputs=input_tensor, outputs=preds)

94683136/94668760 [==============================] - 1s 0us/step


In [9]:
for layer in Concatenated_model.layers:
  layer.trainable= True

#Concatenated_model.summary()

In [ ]:
from keras import regularizers, optimizers

Concatenated_model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001), loss='categorical_crossentropy',metrics=['accuracy'])

filepath="models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%full_name 
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max') #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]

epoch_num = 1
train_df =pd.read_csv("train{}.csv".format(epoch_num)) #Add data generators for parsing images to the network while training
train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_df,
      directory='All/All',
      x_col="filename",
      y_col="class",
      target_size=(300, 300),
      batch_size=20,
      class_mode='categorical',shuffle=True)
validation_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory='All/All',
        x_col="filename",
        y_col="class",
        target_size=(300, 300),
        batch_size=20,
        class_mode='categorical',shuffle=True)
history = Concatenated_model.fit_generator(train_generator, epochs=100,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) #start training



In [ ]:
#After the training is done, we attempt to evaluate our saved networks.

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, len(history.epoch) + 1)


fig_model_performance = plt.figure(figsize=(10,10))
plt.subplot(2,2,1)
plt.plot(epochs_range, acc, label='Training')
plt.plot(epochs_range, val_acc, label='Validation')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.title('Model accuracy')
plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.show()

In [ ]:
plt.plot(epochs_range, loss, label='Training')
plt.plot(epochs_range, val_loss, label='Validation')
plt.legend(loc="best")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.show()

In [ ]:
Concatenated_model.save("first_model.h5")

In [ ]:
from tqdm import tqdm

In [ ]:
trained_models=[]
for r,d,f in os.walk('models'):
  for file in f:
    trained_models.append(os.path.join(r,file)) #Add the saved models in the models directory
reports={}
test_df = pd.read_csv("all_test.csv") #Load the full test CSV file that includes 11302 images
validation_generator = test_datagen.flow_from_dataframe( #Set new data generator
        dataframe=test_df,
        directory='All/All',
        x_col="filename",
        y_col="class",
        target_size=(300, 300),
        batch_size=20,
        class_mode='categorical',shuffle=True)
for trained_model in tqdm(trained_models):
  k.clear_session()
  net=keras.models.load_model(trained_model) #load model
  covid_label= validation_generator.class_indices['COVID-19'] #get the index of COVUD-19 class 
  pneu_label= validation_generator.class_indices['pneumonia'] #get the index of pneumonia class 
  normal_label= validation_generator.class_indices['normal']  #get the index of normal class 
  tp=0 #True Positives
  fp=0 #False Positives
  anum=0 #All the images numbers
  ###########
  wrong_covid=0
  correct_covid=0
  not_detected_covid=0
  covid_num=0
  ###########
  wrong_pneu=0
  correct_pneu=0
  not_detected_pneu=0
  pneu_num=0
  ############
  wrong_normal=0
  correct_normal=0
  not_detected_normal=0
  normal_num=0
  ##############
  wrong_covid_normal=0 #COVID-19 class which has been detected as normal, incorrectly
  wrong_covid_pneu=0   #COVID-19 class which has been detected as pneumonia, incorrectly
  wrong_pneu_covid=0   #pneumonia class which has been detected as COVID-19 , incorrectly
  wrong_pneu_normal=0  #pneumonia class which has been detected as normal , incorrectly
  wrong_normal_pneu=0  #normal class which has been detected as pneumonia , incorrectly
  wrong_normal_covid=0  #normal class which has been detected as COVID-19 , incorrectly
  ################
  for num,img_name in enumerate(validation_generator.filenames): #load image
    gt_ind=validation_generator.classes[num] #get the loaded image class index
    img=cv2.resize(cv2.imread(os.path.join('All','All',img_name)),(300,300)) #resize image
    img=img.astype('float32') / 255.0 #scale the image
    pred_ind=np.argmax(net.predict(np.expand_dims(img,axis=0))[0]) #get the predicted class index

    anum+=1 #count the number of images


    if gt_ind==covid_label:
      covid_num+=1
    if gt_ind==pneu_label:
      pneu_num+=1
    if gt_ind==normal_label:
      normal_num+=1
    ##################
    if gt_ind==covid_label and pred_ind==covid_label: 
      correct_covid+=1
    if gt_ind==covid_label and pred_ind!=covid_label:
      not_detected_covid+=1
      if pred_ind==pneu_label:
        wrong_covid_pneu+=1
      elif pred_ind==normal_label:
        wrong_covid_normal+=1
    if gt_ind!=covid_label and pred_ind==covid_label:
      wrong_covid+=1
    ###########################################
    if gt_ind==normal_label and pred_ind==normal_label: 
      correct_normal+=1
    if gt_ind==normal_label and pred_ind!=normal_label:
      not_detected_normal+=1
      if pred_ind==pneu_label:
        wrong_normal_pneu+=1
      elif pred_ind==covid_label:
        wrong_normal_covid+=1
    if gt_ind!=normal_label and pred_ind==normal_label:
      wrong_normal+=1
    ###########################################
    if gt_ind==pneu_label and pred_ind==pneu_label: 
      correct_pneu+=1
    if gt_ind==pneu_label and pred_ind!=pneu_label:
      not_detected_pneu+=1
      if pred_ind==normal_label:
        wrong_pneu_normal+=1
      elif pred_ind==covid_label:
        wrong_pneu_covid+=1
    if gt_ind!=pneu_label and pred_ind==pneu_label:
      wrong_pneu+=1
    ###########################################

    if pred_ind==gt_ind:
      tp+=1
    else:
      fp+=1

  reports[trained_model[1:]]=[correct_covid,wrong_covid,not_detected_covid,
                   correct_pneu,wrong_pneu,not_detected_pneu,
                   correct_normal,wrong_normal,not_detected_normal,
                   covid_num,pneu_num,normal_num,tp,fp,wrong_covid_normal,wrong_covid_pneu,
                   wrong_pneu_covid,wrong_pneu_normal,wrong_normal_pneu,wrong_normal_covid]


  print(trained_model[1:])
  print('correct_covid: ',correct_covid/covid_num,'%')
  print('wrong_covid: ',wrong_covid)
  print('not_detected_covid: ',not_detected_covid/covid_num,'%')
  print('###########################')

with open('{}-fold{}.csv'.format(trained_model[1:],fold_num),newline='', mode='w') as csvfile: #save the evaluation data into csvfile
     csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
     csv_writer.writerow(['name','correct_covid','wrong_covid','not_detected_covid',
                          'correct_pneu','wrong_pneu','not_detected_pneu',
                          'correct_normal','wrong_normal','not_detected_normal',
                          'covid_num','pneu_num','normal_num','tp','fp','wrong_covid_normal','wrong_covid_pneu',
                  'wrong_pneu_covid','wrong_pneu_normal','wrong_normal_pneu','wrong_normal_covid'])
     for key in reports.keys():
         row=[key]
         row.extend(reports[key])